Aclaración: ejecutamos este modelo con y sin limpieza de texto. El resultado fue mejor para el caso en el que no realizamos la limpieza

#Downloads

In [ ]:
pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 49kB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.2MB 41.3MB/s 
     |████████████████████████████████| 491kB 40.0MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.6MB/s 


In [ ]:
pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 71kB 1.9MB/s 


In [ ]:
!wget --quiet https://github.com/google-research/bert

#Libraries

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
from datetime import datetime

In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [ ]:
import pandas as pd
import re,string
import html
import urllib

#Data

In [ ]:
train = pd.read_csv('train_split.csv')
validation = pd.read_csv('validation_split.csv')
test = pd.read_csv('test_kaggle.csv')

#Preprocess

In [ ]:
def remove_hashtags(text):
  return text.replace('#','')
	
def remove_special_characters(text):
  return text.replace('\n', ' ').replace('å', ' ').replace('ê', ' ').replace('û', ' ').replace('[^\w\s]',' ').replace('÷', ' ').replace('ª', ' ').replace('ì', ' ').replace('¢', ' ').replace('\x89', ' ').replace('\x9d', ' ').replace('ã', ' ').replace('ï', ' ').replace('©', ' ').replace('ó', ' ').replace('ò', ' ').replace('â', ' ').replace('\x9b', ' ').replace('\x8a', ' ').replace('\x93', ' ').replace('\x8f', ' ')\
             .replace('\x8c', ' ').replace('¼', ' ').replace('¥', ' ').replace('\x92', ' ').replace('·', ' ').replace('\x82', ' ').replace('Û', ' ').replace('Ó', ' ').replace('Ê', ' ').replace('Ï', ' ')

def remove_URL(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',str(text))
	
def remove_mention(text):
  return text.replace('@','')

def remove_puntuaction(text):
  return text.translate(str.maketrans('','',string.punctuation))

# # Hago decode de todos los textos que esten encodeados en formato url
def decode_URLformat(text):
  return urllib.parse.unquote(str(text))

def remove_entities(text):
  return html.unescape(text)

def clean_text(text):
  clean_text = remove_hashtags(text)
  clean_text = remove_special_characters(clean_text)
  clean_text = remove_URL(clean_text)
  clean_text = remove_mention(clean_text)
  clean_text = remove_puntuaction(clean_text)
  clean_text = decode_URLformat(clean_text)
  clean_text = remove_entities(clean_text)
  return clean_text

In [ ]:
repeated_tweets = train.groupby('text').agg({'target':['sum','count']}).droplevel(0,axis = 1)
repeated_tweets.columns = ['target_sum','count']

In [ ]:
repeated_tweets = repeated_tweets[repeated_tweets['count'].apply(lambda x: x > 1)]

In [ ]:
#Me quedo con los tweet exactamente iguales, con distintos target que me arruinarian el entrenamiento
repeated_tweets = repeated_tweets[repeated_tweets.apply(lambda x: x['target_sum'] != x['count'] , axis = 1)]

In [ ]:
repeated_tweets = (repeated_tweets.reset_index().text.values)
train = train[train.text.apply(lambda x: not x in repeated_tweets)]

In [ ]:
train.sample(2)

,id,keyword,location,text,target
1206,2169,catastrophic,NaN,The Catastrophic Effects of Hiroshima and Naga...,1
4888,8715,sinking,NaN,In the movie 'Titanic' Jack and Rose both coul...,0


In [ ]:
train['text'] = train['text'].apply(lambda x: clean_text(x))
validation['text'] = validation['text'].apply(lambda x: clean_text(x))
test['text'] = test['text'].apply(lambda x: clean_text(x))

In [ ]:
train.sample(5)

,id,keyword,location,text,target
3685,6543,injury,"Sacramento, CA",Traffic Collision No Injury I5 S at I5 S 43rd...,1
4004,7112,military,"Bozeman, Montana",Army names 10th Mountain units for Iraq Afghan...,0
5179,9247,sunk,NaN,British diver Neil Anthony Fears found dead by...,1
2850,5078,famine,"Edappally,Kochi",Building Muscle With Feast And Famine Feeding ...,0
5664,10132,upheaval,NaN,Acquire your postexistence straight a elevatio...,0


In [ ]:
# guid: ID único. No lo utilizamos en este caso
# text_a: cadena. El texto tokenizado de la primera secuencia. Como no trabajamos con pares de oraciones solo aclaro esta (text_b no se usa)
# text_b: cadena (opcional)
# etiqueta: cadena (opcional). La etiqueta del ejemplo. Esto debería ser especificado para ejemplos de entrenamiento y validacion, pero no para ejemplos de prueba.
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,text_a = x['text'], text_b = None, label = x['target']), axis = 1)
test_InputExamples = validation.apply(lambda x: bert.run_classifier.InputExample(guid=None, text_a = x['text'], text_b = None, label = x['target']), axis = 1)

#Model

In [ ]:
# Modelo BERT que utilizamos
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

* Pasamos a minuscula porque estamos usando un modelo de BERT en minuscula
* Tokenizamos
* Dividir las palabras en Wordpieces (ejemplo: "calling" -> ["call", "##ing"])
* Asignamos indices a las palabras utilizando el modelo de BERT
* Agregamos tokens especiales
* Se añaden los tokens index y segment 

In [ ]:
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

In [ ]:
tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# Longitud maxima de 128 tokens
MAX_SEQ_LENGTH = 128

label_list = [0,1]

# Convierto al formato de entrada de BERT. Utilizo un script de Google team disponible en github
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 6034


INFO:tensorflow:Writing example 0 of 6034


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  #Model bert pre-trained
  bert_module = hub.Module(BERT_MODEL_HUB,trainable=True)

  bert_inputs = dict(input_ids=input_ids,input_mask=input_mask,segment_ids=segment_ids)
  bert_outputs = bert_module(inputs=bert_inputs,signature="tokens",as_dict=True)

  # "pooled_output" para tareas de clasificacion
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  
  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable("output_weights", [num_labels, hidden_size],initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  def model_fn(features, labels, mode, params):  

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(label_ids,predicted_labels)
        auc = tf.metrics.auc(label_ids,predicted_labels)
        recall = tf.metrics.recall(label_ids,predicted_labels)
        precision = tf.metrics.precision(label_ids,predicted_labels) 
        true_pos = tf.metrics.true_positives(label_ids,predicted_labels)
        true_neg = tf.metrics.true_negatives(label_ids,predicted_labels)   
        false_pos = tf.metrics.false_positives(label_ids,predicted_labels)  
        false_neg = tf.metrics.false_negatives(label_ids,predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [ ]:
# Compute train and warmup steps from batch size
# Hyperparameters are copied from (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
# Specify outpit directory and number of checkpoint steps to save

OUTPUT_DIR = 'model_saved'

run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'model_saved', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa4777b1dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'model_saved', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa4777b1dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 565 into model_saved/model.ckpt.


INFO:tensorflow:Saving checkpoints for 565 into model_saved/model.ckpt.


INFO:tensorflow:Loss for final step: 0.030470196.


INFO:tensorflow:Loss for final step: 0.030470196.


Training took time  7:34:00.970020


In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 565: auc = 0.8141764, eval_accuracy = 0.8266579, f1_score = 0.7807309, false_negatives = 167.0, false_positives = 97.0, global_step = 565, loss = 0.5156872, precision = 0.8289242, recall = 0.7378336, true_negatives = 789.0, true_positives = 470.0


INFO:tensorflow:Saving dict for global step 565: auc = 0.8141764, eval_accuracy = 0.8266579, f1_score = 0.7807309, false_negatives = 167.0, false_positives = 97.0, global_step = 565, loss = 0.5156872, precision = 0.8289242, recall = 0.7378336, true_negatives = 789.0, true_positives = 470.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 565: model_saved/model.ckpt-565


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 565: model_saved/model.ckpt-565


{'auc': 0.8141764,
 'eval_accuracy': 0.8266579,
 'f1_score': 0.7807309,
 'false_negatives': 167.0,
 'false_positives': 97.0,
 'global_step': 565,
 'loss': 0.5156872,
 'precision': 0.8289242,
 'recall': 0.7378336,
 'true_negatives': 789.0,
 'true_positives': 470.0}

In [ ]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences]
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
pred_sentences = test['text'].values
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 3263


INFO:tensorflow:Writing example 0 of 3263


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from model_saved/model.ckpt-565


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [ ]:
labels_predicted = []
tweets = []

for prediction in predictions:
  tweets.append(prediction[0])
  labels_predicted.append(prediction[2])

In [ ]:
labels_test = []

for x in labels_predicted:
  if (x == 'Positive'): labels_test.append(1)
  else: labels_test.append(0)

In [ ]:
predictions_df = (pd.DataFrame({'id':test.id.values,'target':labels_test}))

In [ ]:
# Empeora resultado. Fue mejor resultado sin limpieza de texto
predictions_df.to_csv('bert_fine_tuning_2.csv',header=True,index=False)